# **1. Configuración del Ambiente**


---

In [40]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
global df_traffic, resultados, modelo, modelo_clasificacion

# **2. Creación de Modelo de Gradient Boosted Regressor**


---

In [41]:
df_traffic = pd.read_csv('Classifier.csv', sep=';')
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   visitNumber         12283 non-null  int64  
 1   browser             12283 non-null  int64  
 2   operatingSystem     12283 non-null  int64  
 3   deviceCategory      12283 non-null  int64  
 4   continent           12283 non-null  int64  
 5   country             12283 non-null  int64  
 6   city                12283 non-null  int64  
 7   networkDomain       12283 non-null  int64  
 8   source              12283 non-null  int64  
 9   medium              12283 non-null  int64  
 10  keyword             12283 non-null  int64  
 11  referralPath        12283 non-null  int64  
 12  adContent           12283 non-null  int64  
 13  pageviews           12283 non-null  int64  
 14  bounces             12283 non-null  float64
 15  newVisits           12283 non-null  float64
 16  tran

In [42]:
# Dividir los datos en conjunto de entrenamiento y prueba
X = df_traffic.drop(columns=['transactionRevenue'])
y = df_traffic['transactionRevenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 503.5913893772894


In [43]:
#definiendo el K - número de subconjuntos
cv = KFold(n_splits = 5, shuffle=True)

In [44]:
accuracy_cross_val = cross_val_score(estimator = decision_tree,
                                       X = X_train,
                                       y = y_train,
                                       scoring = 'neg_mean_squared_error',
                                       cv = cv)
print('Accuracy promedio de mse: {}'.format(round(accuracy_cross_val.mean(), 3)))

Accuracy promedio de mse: -549.835


In [45]:
# Selección de hiperparámetros para la regresión lineal
param_decision_tree = {
    'criterion': ['mse', 'friedman_mse', 'mae'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None]
}


In [46]:
# Configurar RandomizedSearchCV
randomized_search_dtm = RandomizedSearchCV(
    estimator=decision_tree,
    param_distributions=param_decision_tree,
    n_iter=10,  
    cv=5, 
    scoring='neg_mean_squared_error',  
    n_jobs=-1, 
    random_state=42  
)


In [47]:
# Ajustar el modelo
randomized_search_dtm.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
                   param_distributions={'criterion': ['mse', 'friedman_mse',
                                                      'mae'],
                                        'max_depth': [None, 5, 10, 20],
                                        'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'splitter': ['best', 'random']},
                   random_state=42, scoring='neg_mean_squared_error')

In [48]:
# Obtener los mejores parámetros y la mejor puntuación
best_params_dtm = randomized_search_dtm.best_params_
best_score_dtm = randomized_search_dtm.best_score_

print("Mejores parámetros:", best_params_dtm)
print("Mejor puntuación:", best_score_dtm)

Mejores parámetros: {'splitter': 'random', 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 10, 'criterion': 'friedman_mse'}
Mejor puntuación: -396.5940835202558


In [49]:
# Crear una nueva instancia de GradientBoostingRegressor con los mejores parámetros encontrados
best_dt_model = DecisionTreeRegressor(**randomized_search_dtm.best_params_)

# Entrenar el modelo con los datos de entrenamiento
best_dt_model.fit(X_train, y_train)

# Predecir con el modelo entrenado
y_pred = best_dt_model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calcular R^2
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calcular RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Mean Squared Error: 264.0897690394761
R^2 Score: 0.3849754203331497
Root Mean Squared Error: 16.25083902570806


In [50]:
model_scores = {
    'model': 'DecisionTreeRegressor-Edwin6',
    'mse': mse,
    'r2': r2,
    'rmse': rmse
}
print(model_scores)

{'model': 'DecisionTreeRegressor-Edwin6', 'mse': 264.0897690394761, 'r2': 0.3849754203331497, 'rmse': 16.25083902570806}


In [51]:
def save_scores(model_scores: dict):
  status = []
  #Utilizamos un try-except en caso de que el archivo no exista que cree uno
  try:
    #Si el archivo existe continúa por acá
    scores = pd.read_csv('scores.csv', sep=';')
    status.append('El archivo existe')
    exist = model_scores['model'] in scores['model'].values
    #Validamos si el modelo ya existe en el archivo de scores
    if (exist):
      #Si el modelo ya existe reemplazamos sus valores
      status.append('El modelo existe')
      criteria = scores['model'] == model_scores['model']
      index = scores[criteria].index[0]
      scores.iloc[index] = model_scores
      status.append('Se reemplazaron los valores del modelo')
    else:
      #Si el modelo no existe lo agregamos
      status.append('El modelo no existe')
      df_model_scores = pd.DataFrame(model_scores, index=[0])
      scores = pd.concat([scores, df_model_scores], ignore_index=True)
      status.append('Se añadió modelo nuevo y sus valores')
  except:
    #Si no existe el archivo lo creamos y cargamos los scores del modelo
    status.append('El archivo no existe')
    scores = pd.DataFrame(model_scores, index=[0])

  status.append('Se sobrescrive el archivo scores.csv con valores nuevos')
  scores.to_csv('scores.csv', sep=';', index=False)
  return status

In [52]:
save_scores(model_scores)

['El archivo existe',
 'El modelo no existe',
 'Se añadió modelo nuevo y sus valores',
 'Se sobrescrive el archivo scores.csv con valores nuevos']